In [10]:
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

cards_df = pd.read_csv(
    "../../data/processed/analise_cartoes.csv"
)  # ou substitua por `df = seu_dataframe`
cards_df.info()
TEAM_COL = "clube"
POSITION_COL = "posicao"
MINUTE_COL = "tempo_jogo"
CARDS_COL = "Quantidade_Cartoes"

times = ["Fortaleza", "Botafogo-RJ", "Corinthians", "Palmeiras", "Flamengo", "Sao Paulo", "Internacional",
         "Atletico-MG", "Cruzeiro", "Gremio", "Vasco", "Bahia", "Fluminense",
         "Athletico-PR", "Vitoria", "Cuiaba", "Atletico-GO", "Criciuma", "Juventude", "Bragantino"]

df = cards_df[cards_df['clube'].isin(times)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1870 entries, 0 to 1869
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   posicao             1870 non-null   object
 1   tempo_jogo          1870 non-null   object
 2   clube               1870 non-null   object
 3   Quantidade_Cartoes  1870 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 58.6+ KB


In [11]:
import plotly.express as px
import pandas as pd
import numpy as np

# Ensure the dataset is loaded
cards_df = pd.read_csv("../../data/processed/analise_cartoes.csv")

# Define the order of the categories for 'tempo_jogo'
tempo_order = ['00-10', '11-20', '21-30', '31-40', '41-45', '45+', '46-60', '61-70', '71-80', '81-90', '90+']

# Convert 'tempo_jogo' to a categorical variable with the defined order
cards_df['tempo_jogo'] = pd.Categorical(cards_df['tempo_jogo'], categories=tempo_order, ordered=True)

# Filter the dataset for specific teams
times = ["Fortaleza", "Botafogo-RJ", "Corinthians", "Palmeiras", "Flamengo", "Sao Paulo", "Internacional",
         "Atletico-MG", "Cruzeiro", "Gremio", "Vasco", "Bahia", "Fluminense",
         "Athletico-PR", "Vitoria", "Cuiaba", "Atletico-GO", "Criciuma", "Juventude", "Bragantino"]
df = cards_df[cards_df['clube'].isin(times)]

# Create the bubble chart
fig = px.scatter(df,
                 x=MINUTE_COL,
                 y=TEAM_COL,
                 size=CARDS_COL,
                 color=POSITION_COL,
                 title='Bubble Chart com Facetas: Minuto × Cartões por Clube e Posição',
                 labels={'tempo_jogo': 'Minuto do Jogo', 'Quantidade_Cartoes': 'Quantidade de Cartões'},
                 height=800,
                 width=800,
                 opacity=0.5,
                 category_orders={MINUTE_COL: tempo_order, TEAM_COL: sorted(df[TEAM_COL].unique())})

# Update marker styles
fig.update_layout(scattermode='group',scattergap=1)
#fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')))

# Show the chart
fig.show()

In [50]:
import plotly.graph_objects as go

# Create the bubble chart
fig = go.Figure()

# Loop through each position and create a separate trace
for position in df[POSITION_COL].unique():
    subset = df[df[POSITION_COL] == position]
    
    fig.add_trace(go.Scatter(
        x=subset[MINUTE_COL],
        y=subset[TEAM_COL],
        name=position,  # Legend entry for the position
        mode='markers',
        marker=dict(
            size=subset[CARDS_COL]/2,  # Scale the bubble size
            sizemode='diameter',
            line=dict(width=1, color='DarkSlateGrey')  # Add a border to the bubbles
        ),
        opacity=0.7  # Set opacity for better visualization of overlapping bubbles
    ))

# Update the layout to ensure proper grouping and spacing
fig.update_layout(
    title='Bubble Chart: Minuto × Cartões por Clube e Posição',
    height=800,
    width=1000,
    xaxis=dict(
        title='Minuto do Jogo',
        type='category',
        categoryorder='array',
        categoryarray=tempo_order  # Ensure the correct order of 'tempo_jogo'
    ),
    yaxis=dict(
        title='Clube',
        type='category',
        categoryorder='category ascending'  # Ensure clubs are sorted alphabetically
    ),
    legend=dict(
        title='Posição',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    scattermode='group',scattergap=1
)

# Show the chart
fig.show()

In [ ]:
# Source - https://stackoverflow.com/a
# Posted by Derek O
# Retrieved 2025-11-24, License - CC BY-SA 4.0

import pandas as pd
import plotly.graph_objects as go

cards_df = pd.read_csv("../../data/processed/analise_cartoes.csv")

# Define the order of the categories for 'tempo_jogo'
tempo_order = ['00-10', '11-20', '21-30', '31-40', '41-45', '45+', '46-60', '61-70', '71-80', '81-90', '90+']

# Convert 'tempo_jogo' to a categorical variable with the defined order
cards_df['tempo_jogo'] = pd.Categorical(cards_df['tempo_jogo'], categories=tempo_order, ordered=True)

# Filter the dataset for specific teams
times = ["Fortaleza", "Botafogo-RJ", "Corinthians", "Palmeiras", "Flamengo", "Sao Paulo", "Internacional",
         "Atletico-MG", "Cruzeiro", "Gremio", "Vasco", "Bahia", "Fluminense",
         "Athletico-PR", "Vitoria", "Cuiaba", "Atletico-GO", "Criciuma", "Juventude", "Bragantino"]
df = cards_df[cards_df['clube'].isin(times)]

time = {'00-10':1, '11-20':2, '21-30':3, '31-40':4, '41-45':5, '45+':6, '46-60':7, '61-70':8, '71-80':9, '81-90':10, '90+':11}
jitter = {'Atacante': -0.15, 'Zagueiro': -0.05, 'Desconhecido':0,'Goleiro': 0.05, 'Meio-campo': 0.15}

fig = go.Figure()
## position 1 of each row is Person... position 4 is the Energy value
s = df.posicao.shift() != df.posicao
name_changes = s[s].index.values
for row in df.itertuples():
    if row[0] in name_changes:
        fig.add_trace(go.Scatter(
            x=[time[row[2]] + jitter[row[1]]],
            y=[row[4]],
            legendgroup=row[1],
            name=row[1],
            mode='markers',
            marker_color=jitter[row[1]],
            showlegend=True
        ))
    else:
        fig.add_trace(go.Scatter(
            x=[time[row[2]] + jitter[row[1]]],
            y=[row[4]],
            legendgroup=row[1],
            name=row[1],
            mode='markers',
            marker_color=jitter[row[1]],
            showlegend=False
        ))

fig.update_traces(marker=dict(size=12,line=dict(width=2,color='DarkSlateGrey')))
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=list(time.values()),
        ticktext=list(time.keys())
    )
)
fig.show() 


In [41]:
import pandas as pd
import plotly.graph_objects as go

# Load the dataset
cards_df = pd.read_csv("../../data/processed/analise_cartoes.csv")

# Define the order of the categories for 'tempo_jogo'
tempo_order = ['00-10', '11-20', '21-30', '31-40', '41-45', '45+', '46-60', '61-70', '71-80', '81-90', '90+']

# Convert 'tempo_jogo' to a categorical variable with the defined order
cards_df['tempo_jogo'] = pd.Categorical(cards_df['tempo_jogo'], categories=tempo_order, ordered=True)

# Filter the dataset for specific teams
times = ["Fortaleza", "Botafogo-RJ", "Corinthians", "Palmeiras", "Flamengo", "Sao Paulo", "Internacional",
         "Atletico-MG", "Cruzeiro", "Gremio", "Vasco", "Bahia", "Fluminense",
         "Athletico-PR", "Vitoria", "Cuiaba", "Atletico-GO", "Criciuma", "Juventude", "Bragantino"]
df = cards_df[cards_df['clube'].isin(times)]

# Map 'posicao' to numeric values for coloring
position_colors = {pos: i for i, pos in enumerate(df['posicao'].unique())}
df['posicao_color'] = df['posicao'].map(position_colors)

# Create a mapping for jitter to avoid overlapping bubbles
jitter = {'Goleiro': -0.20, 'Desconhecido': -0.10, 'Atacante': 0, 'Meio-campo': 0.15, 'Zagueiro': 0.35}
color = {'Atacante': 'red', 'Zagueiro': 'blue', 'Desconhecido': 'gray', 'Goleiro': 'green', 'Meio-campo': 'orange'}
# Create the bubble chart
fig = go.Figure()

for position in df['posicao'].unique():
    subset = df[df['posicao'] == position]
    fig.add_trace(go.Scatter(
        x=[tempo_order.index(minute) + jitter[position] for minute in subset['tempo_jogo']],
        y=subset['clube'],
        mode='markers',
        name=position,
        marker=dict(
            size=subset['Quantidade_Cartoes'],  # Bubble size based on Quantidade_Cartoes
            sizemode='area',
            color=jitter[position],  # Use numeric color mapping
            line=dict(width=1, color='DarkSlateGrey')
        ),
        hovertemplate=(
            "Clube: %{y}<br>"
            "Minuto: %{x}<br>"
            "Cartões: %{marker.size}<br>"
            "Posição: %{name}<extra></extra>"
        )
    ))

# Update layout for better visualization
fig.update_layout(
    title="Bubble Chart: Minuto × Cartões por Clube e Posição",
    xaxis=dict(
        title="Minuto do Jogo",
        tickmode="array",
        tickvals=list(range(len(tempo_order))),
        ticktext=tempo_order
    ),
    yaxis=dict(
        title="Clube",
        categoryorder="category ascending"
    ),
    legend=dict(
        title="Posição",
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    height=800,
    width=1200
)

# Show the chart
fig.show()

C:\Users\david\AppData\Local\Temp\ipykernel_19976\714586326.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
import pandas as pd
import plotly.graph_objects as go

# Load the dataset
cards_df = pd.read_csv("../../data/processed/analise_cartoes.csv")

# Define the order of the categories for 'tempo_jogo'
tempo_order = ['00-10', '11-20', '21-30', '31-40', '41-45', '45+', '46-60', '61-70', '71-80', '81-90', '90+']

# Convert 'tempo_jogo' to a categorical variable with the defined order
cards_df['tempo_jogo'] = pd.Categorical(cards_df['tempo_jogo'], categories=tempo_order, ordered=True)

# Filter the dataset for specific teams
times = ["Fortaleza", "Botafogo-RJ", "Corinthians", "Palmeiras", "Flamengo", "Sao Paulo", "Internacional",
         "Atletico-MG", "Cruzeiro", "Gremio", "Vasco", "Bahia", "Fluminense",
         "Athletico-PR", "Vitoria", "Cuiaba", "Atletico-GO", "Criciuma", "Juventude", "Bragantino"]
df = cards_df[cards_df['clube'].isin(times)]

# Create a mapping for jitter to avoid overlapping bubbles
jitter = {'Goleiro': -0.20, 'Desconhecido': -0.10, 'Atacante': 0, 'Meio-campo': 0.15, 'Zagueiro': 0.35}
# Define specific colors for positions
color_map = {'Atacante': 'red', 'Zagueiro': 'blue', 'Desconhecido': 'gray', 'Goleiro': 'green', 'Meio-campo': 'orange'}

# Scale factor for bubbles
SIZE_FACTOR = 5

fig = go.Figure()

# Add traces for each position
for position in df['posicao'].unique():
    subset = df[df['posicao'] == position]
    fig.add_trace(go.Scatter(
        x=[tempo_order.index(minute) + jitter[position] for minute in subset['tempo_jogo']],
        y=subset['clube'],
        mode='markers',
        name=position,
        legendgroup='group1',
        legendgrouptitle_text="Posição",
        marker=dict(
            size=subset['Quantidade_Cartoes'] * SIZE_FACTOR,  # Increase bubble size
            sizemode='area',
            color=color_map.get(position, 'black'),
            line=dict(width=1, color='DarkSlateGrey')
        ),
        hovertemplate=(
            "Clube: %{y}<br>"
            "Minuto: %{x}<br>"
            "Cartões: %{marker.size}<br>"
            "Posição: %{name}<extra></extra>"
        )
    ))

# Add dummy traces for size legend
sizes = [1, 10, 20, 30, 40, 50]  # Example card quantities to show in legend
for s in sizes:
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        name=f'{s} Cartões',
        legendgroup='group2',
        legendgrouptitle_text="Qtd. Cartões",
        marker=dict(
            size=s * SIZE_FACTOR,
            sizemode='area',
            color='grey',
        )
    ))

# Update layout
fig.update_layout(
    title="Bubble Chart: Minuto × Cartões por Clube e Posição",
    xaxis=dict(
        title="Minuto do Jogo",
        tickmode="array",
        tickvals=list(range(len(tempo_order))),
        ticktext=tempo_order
    ),
    yaxis=dict(
        title="Clube",
        categoryorder="category ascending"
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    height=800,
    width=1200
)

# Show the chart
fig.show()